# 📊 Business Metrics - Excel → SQLite 변환 및 서버 업로드

이 노트북은 구글 드라이브의 엑셀 파일을 SQLite로 변환하고 서버에 업로드합니다.

## 워크플로우
1. 구글 드라이브 마운트
2. 엑셀 파일 읽기 (2024, 2025년 데이터)
3. SQLite DB로 변환
4. 서버로 업로드

## 1. 설정

In [ ]:
# 필요한 패키지 설치
!pip install -q openpyxl requests

In [ ]:
import sqlite3
import os
import json
from pathlib import Path
from openpyxl import load_workbook
import requests
from google.colab import drive
import time

In [ ]:
# ============ 설정 ============
# 서버 설정 (본 사이트 또는 데모 선택)
SERVER_CONFIG = {
    'main': {
        'url': 'http://14.7.14.31:6001/api/upload-db',
        'name': '본 사이트'
    },
    'demo': {
        'url': 'http://14.7.14.31:6005/api/upload-db',
        'name': '데모 사이트'
    }
}

# 사용할 서버 선택 ('main' 또는 'demo')
USE_SERVER = 'demo'  # 먼저 데모에서 테스트 권장

# API 키 (서버와 동일하게 설정)
API_KEY = 'biofl1411-upload-key'

# 드라이브 내 데이터 경로 설정
DRIVE_BASE_PATH = '/content/drive/MyDrive/business_metrics_data'

# 출력 DB 파일 경로
OUTPUT_DB = '/content/business_data.db'

print(f"서버: {SERVER_CONFIG[USE_SERVER]['name']}")
print(f"URL: {SERVER_CONFIG[USE_SERVER]['url']}")
print(f"드라이브 경로: {DRIVE_BASE_PATH}")

## 2. 구글 드라이브 마운트

In [ ]:
# 구글 드라이브 마운트
drive.mount('/content/drive')

# 데이터 폴더 확인
base_path = Path(DRIVE_BASE_PATH)
if not base_path.exists():
    print(f"⚠️ 경로가 없습니다: {DRIVE_BASE_PATH}")
    print("드라이브에 다음 폴더 구조를 만들어주세요:")
    print(f"  {DRIVE_BASE_PATH}/2024/*.xlsx (메인 데이터)")
    print(f"  {DRIVE_BASE_PATH}/2025/*.xlsx (메인 데이터)")
    print(f"  {DRIVE_BASE_PATH}/food_item/2024/*.xlsx (음식항목 데이터)")
    print(f"  {DRIVE_BASE_PATH}/food_item/2025/*.xlsx (음식항목 데이터)")
else:
    print(f"✅ 데이터 경로 확인됨: {DRIVE_BASE_PATH}")
    # 하위 폴더 확인
    for folder in ['2024', '2025', 'food_item/2024', 'food_item/2025']:
        folder_path = base_path / folder
        if folder_path.exists():
            files = list(folder_path.glob('*.xlsx'))
            print(f"  📁 {folder}: {len(files)}개 파일")
        else:
            print(f"  ⚠️ {folder}: 폴더 없음")

## 3. SQLite DB 초기화

In [ ]:
def init_sqlite_db(db_path):
    """SQLite 데이터베이스 초기화"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # 기본 데이터 테이블 (excel_data)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS excel_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            year TEXT,
            접수번호 TEXT,
            접수일자 TEXT,
            발행일 TEXT,
            검체유형 TEXT,
            업체명 TEXT,
            의뢰인명 TEXT,
            업체주소 TEXT,
            영업담당 TEXT,
            검사목적 TEXT,
            총금액 REAL,
            시험분야 TEXT,
            입금일 TEXT,
            입금여부 TEXT,
            검사구분 TEXT,
            입금구분 TEXT,
            업체분류 TEXT,
            raw_data TEXT
        )
    ''')

    # food_item 데이터 테이블
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS food_item_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            year TEXT,
            접수일자 TEXT,
            발행일 TEXT,
            검체유형 TEXT,
            업체명 TEXT,
            의뢰인명 TEXT,
            업체주소 TEXT,
            항목명 TEXT,
            규격 TEXT,
            항목담당 TEXT,
            결과입력자 TEXT,
            입력일 TEXT,
            분석일 TEXT,
            항목단위 TEXT,
            시험결과 TEXT,
            시험치 TEXT,
            성적서결과 TEXT,
            판정 TEXT,
            검사목적 TEXT,
            긴급여부 TEXT,
            항목수수료 REAL,
            영업담당 TEXT
        )
    ''')

    # 메타데이터 테이블
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS file_metadata (
            file_path TEXT PRIMARY KEY,
            mtime REAL,
            row_count INTEGER
        )
    ''')

    # 토큰 사용량 테이블
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS token_usage (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT,
            year_month TEXT,
            model TEXT,
            input_tokens INTEGER,
            output_tokens INTEGER,
            total_tokens INTEGER,
            cost_usd REAL,
            cost_krw REAL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')

    # 인덱스 생성
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_excel_year ON excel_data(year)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_excel_manager ON excel_data(영업담당)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_excel_purpose ON excel_data(검사목적)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_food_year ON food_item_data(year)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_food_manager ON food_item_data(영업담당)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_food_purpose ON food_item_data(검사목적)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_food_item ON food_item_data(항목명)')

    conn.commit()
    conn.close()
    print(f"✅ DB 초기화 완료: {db_path}")

## 4. Excel → SQLite 변환 함수

In [ ]:
def convert_excel_data(conn, base_path, years=['2024', '2025']):
    """메인 Excel 데이터를 SQLite로 변환"""
    cursor = conn.cursor()
    total_records = 0

    for year in years:
        data_path = Path(base_path) / str(year)
        if not data_path.exists():
            print(f"⚠️ {year}년 폴더 없음: {data_path}")
            continue

        files = sorted(data_path.glob("*.xlsx"))
        if not files:
            print(f"⚠️ {year}년 엑셀 파일 없음")
            continue

        print(f"\n📁 {year}년 데이터 변환 시작 ({len(files)}개 파일)")

        # 기존 데이터 삭제
        cursor.execute('DELETE FROM excel_data WHERE year = ?', (year,))

        for f in files:
            try:
                wb = load_workbook(f, read_only=True, data_only=True)
                ws = wb.active
                headers = [cell.value for cell in ws[1]]

                batch = []
                for row_data in ws.iter_rows(min_row=2, values_only=True):
                    row_dict = dict(zip(headers, row_data))
                    batch.append((
                        year,
                        str(row_dict.get('접수번호', '') or ''),
                        str(row_dict.get('접수일자', '') or ''),
                        str(row_dict.get('발행일', '') or ''),
                        str(row_dict.get('검체유형', '') or ''),
                        str(row_dict.get('업체명', '') or ''),
                        str(row_dict.get('의뢰인명', '') or ''),
                        str(row_dict.get('업체주소', '') or ''),
                        str(row_dict.get('영업담당', '') or ''),
                        str(row_dict.get('검사목적', '') or ''),
                        float(row_dict.get('총금액', 0) or 0),
                        str(row_dict.get('시험분야', '') or ''),
                        str(row_dict.get('입금일', '') or ''),
                        str(row_dict.get('입금여부', '') or ''),
                        str(row_dict.get('검사구분', '') or ''),
                        str(row_dict.get('입금구분', '') or ''),
                        str(row_dict.get('업체분류', '') or ''),
                        json.dumps(row_dict, ensure_ascii=False, default=str)
                    ))

                cursor.executemany('''
                    INSERT INTO excel_data
                    (year, 접수번호, 접수일자, 발행일, 검체유형, 업체명, 의뢰인명, 업체주소,
                     영업담당, 검사목적, 총금액, 시험분야, 입금일, 입금여부, 검사구분, 입금구분, 업체분류, raw_data)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', batch)

                # 메타데이터 저장
                cursor.execute('''
                    INSERT OR REPLACE INTO file_metadata (file_path, mtime, row_count)
                    VALUES (?, ?, ?)
                ''', (str(f), f.stat().st_mtime, len(batch)))

                wb.close()
                total_records += len(batch)
                print(f"  ✓ {f.name}: {len(batch):,}건")

            except Exception as e:
                print(f"  ✗ {f.name}: {e}")

    conn.commit()
    return total_records

In [ ]:
def convert_food_item_data(conn, base_path, years=['2024', '2025']):
    """food_item Excel 데이터를 SQLite로 변환"""
    cursor = conn.cursor()
    total_records = 0

    required_columns = ['접수일자', '발행일', '검체유형', '업체명', '의뢰인명', '업체주소',
                        '항목명', '규격', '항목담당', '결과입력자', '입력일', '분석일',
                        '항목단위', '시험결과', '시험치', '성적서결과', '판정', '검사목적',
                        '긴급여부', '항목수수료', '영업담당']

    for year in years:
        food_path = Path(base_path) / "food_item" / str(year)
        if not food_path.exists():
            print(f"⚠️ food_item {year}년 폴더 없음: {food_path}")
            continue

        files = sorted(food_path.glob("*.xlsx"))
        if not files:
            print(f"⚠️ food_item {year}년 엑셀 파일 없음")
            continue

        print(f"\n📁 food_item {year}년 데이터 변환 시작 ({len(files)}개 파일)")

        # 기존 데이터 삭제
        cursor.execute('DELETE FROM food_item_data WHERE year = ?', (year,))

        for f in files:
            try:
                wb = load_workbook(f, read_only=True, data_only=True)
                ws = wb.active
                headers = [cell.value for cell in ws[1]]

                # 컬럼 인덱스 매핑
                col_indices = {}
                for i, h in enumerate(headers):
                    if h in required_columns:
                        col_indices[h] = i

                batch = []
                for row_data in ws.iter_rows(min_row=2, values_only=True):
                    row_dict = {}
                    for col_name, idx in col_indices.items():
                        row_dict[col_name] = row_data[idx] if idx < len(row_data) else None

                    batch.append((
                        year,
                        str(row_dict.get('접수일자', '') or ''),
                        str(row_dict.get('발행일', '') or ''),
                        str(row_dict.get('검체유형', '') or ''),
                        str(row_dict.get('업체명', '') or ''),
                        str(row_dict.get('의뢰인명', '') or ''),
                        str(row_dict.get('업체주소', '') or ''),
                        str(row_dict.get('항목명', '') or ''),
                        str(row_dict.get('규격', '') or ''),
                        str(row_dict.get('항목담당', '') or ''),
                        str(row_dict.get('결과입력자', '') or ''),
                        str(row_dict.get('입력일', '') or ''),
                        str(row_dict.get('분석일', '') or ''),
                        str(row_dict.get('항목단위', '') or ''),
                        str(row_dict.get('시험결과', '') or ''),
                        str(row_dict.get('시험치', '') or ''),
                        str(row_dict.get('성적서결과', '') or ''),
                        str(row_dict.get('판정', '') or ''),
                        str(row_dict.get('검사목적', '') or ''),
                        str(row_dict.get('긴급여부', '') or ''),
                        float(row_dict.get('항목수수료', 0) or 0),
                        str(row_dict.get('영업담당', '') or '')
                    ))

                cursor.executemany('''
                    INSERT INTO food_item_data
                    (year, 접수일자, 발행일, 검체유형, 업체명, 의뢰인명, 업체주소, 항목명, 규격,
                     항목담당, 결과입력자, 입력일, 분석일, 항목단위, 시험결과, 시험치, 성적서결과,
                     판정, 검사목적, 긴급여부, 항목수수료, 영업담당)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', batch)

                # 메타데이터 저장
                cursor.execute('''
                    INSERT OR REPLACE INTO file_metadata (file_path, mtime, row_count)
                    VALUES (?, ?, ?)
                ''', (str(f), f.stat().st_mtime, len(batch)))

                wb.close()
                total_records += len(batch)
                print(f"  ✓ {f.name}: {len(batch):,}건")

            except Exception as e:
                print(f"  ✗ {f.name}: {e}")

    conn.commit()
    return total_records

## 5. 변환 실행

In [ ]:
# DB 초기화
if os.path.exists(OUTPUT_DB):
    os.remove(OUTPUT_DB)
    print(f"기존 DB 삭제: {OUTPUT_DB}")

init_sqlite_db(OUTPUT_DB)

# DB 연결
conn = sqlite3.connect(OUTPUT_DB)

print("\n" + "="*50)
print("Excel → SQLite 변환 시작")
print("="*50)

start_time = time.time()

# 메인 데이터 변환
excel_records = convert_excel_data(conn, DRIVE_BASE_PATH)

# food_item 데이터 변환
food_records = convert_food_item_data(conn, DRIVE_BASE_PATH)

conn.close()

elapsed = time.time() - start_time

print("\n" + "="*50)
print("변환 완료!")
print("="*50)
print(f"excel_data: {excel_records:,}건")
print(f"food_item_data: {food_records:,}건")
print(f"소요 시간: {elapsed:.1f}초")
print(f"DB 파일 크기: {os.path.getsize(OUTPUT_DB) / 1024 / 1024:.1f} MB")

## 6. DB 검증

In [ ]:
# 변환된 DB 검증
conn = sqlite3.connect(OUTPUT_DB)
cursor = conn.cursor()

print("=== 테이블 목록 ===")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = [row[0] for row in cursor.fetchall()]
for t in tables:
    print(f"  - {t}")

print("\n=== 데이터 수 ===")
for table_name in ['excel_data', 'food_item_data']:
    try:
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        print(f"  {table_name}: {cursor.fetchone()[0]:,}건")
    except Exception as e:
        print(f"  {table_name}: {e}")

print("\n=== 연도별 데이터 ===")
for table_name in ['excel_data', 'food_item_data']:
    try:
        cursor.execute(f"SELECT year, COUNT(*) FROM {table_name} GROUP BY year")
        rows = cursor.fetchall()
        for year, count in rows:
            print(f"  {table_name} {year}년: {count:,}건")
    except:
        pass

conn.close()

## 7. 서버 업로드

In [ ]:
def upload_to_server(db_path, server_key='demo', api_key=API_KEY):
    """SQLite DB를 서버에 업로드"""
    server = SERVER_CONFIG[server_key]
    url = server['url']
    name = server['name']

    print(f"\n🚀 {name}에 업로드 중...")
    print(f"   URL: {url}")

    try:
        with open(db_path, 'rb') as f:
            files = {'file': ('business_data.db', f, 'application/octet-stream')}
            headers = {'X-API-Key': api_key}

            response = requests.post(url, files=files, headers=headers, timeout=300)

            if response.status_code == 200:
                result = response.json()
                print(f"\n✅ 업로드 성공!")
                print(f"   메시지: {result.get('message', '')}")
                if 'tables' in result:
                    print("   테이블별 데이터:")
                    for table, count in result['tables'].items():
                        print(f"     - {table}: {count:,}건")
                return True
            elif response.status_code == 401:
                print(f"\n❌ 인증 실패 (401)")
                print(f"   API 키를 확인하세요.")
            else:
                print(f"\n❌ 업로드 실패 ({response.status_code})")
                print(f"   응답: {response.text}")

    except requests.exceptions.Timeout:
        print(f"\n❌ 타임아웃 - 파일이 크거나 네트워크가 느립니다.")
    except requests.exceptions.ConnectionError as e:
        print(f"\n❌ 연결 실패 - 서버에 접속할 수 없습니다.")
        print(f"   {e}")
    except Exception as e:
        print(f"\n❌ 오류: {e}")

    return False

In [ ]:
# 데모 서버에 업로드 (먼저 테스트)
upload_to_server(OUTPUT_DB, server_key='demo')

In [ ]:
# 본 사이트에 업로드 (데모 테스트 후 실행)
# upload_to_server(OUTPUT_DB, server_key='main')

## 8. 드라이브에 DB 백업 (선택)

In [ ]:
# 변환된 DB를 드라이브에도 백업
import shutil
from datetime import datetime

backup_dir = Path(DRIVE_BASE_PATH) / 'db_backup'
backup_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
backup_path = backup_dir / f'business_data_{timestamp}.db'

shutil.copy(OUTPUT_DB, backup_path)
print(f"✅ DB 백업 완료: {backup_path}")

# 오래된 백업 정리 (최근 5개만 유지)
backups = sorted(backup_dir.glob('business_data_*.db'), reverse=True)
for old_backup in backups[5:]:
    old_backup.unlink()
    print(f"🗑️ 오래된 백업 삭제: {old_backup.name}")

---
## 사용 방법

### 1. 구글 드라이브 폴더 구조 설정
```
MyDrive/
└── business_metrics_data/
    ├── 2024/
    │   ├── 1월.xlsx
    │   ├── 2월.xlsx
    │   └── ...
    ├── 2025/
    │   └── ...
    └── food_item/
        ├── 2024/
        │   └── ...
        └── 2025/
            └── ...
```

### 2. 실행 순서
1. 셀 1-4: 설정 및 드라이브 마운트
2. 셀 5: 변환 실행
3. 셀 6: 데이터 검증
4. 셀 7: 서버 업로드 (데모 먼저!)
5. 셀 8: 드라이브 백업 (선택)

### 3. 주의사항
- **항상 데모 서버에서 먼저 테스트** 후 본 사이트에 업로드
- API 키가 맞는지 확인 (기본값: `biofl1411-upload-key`)
- 대용량 파일은 변환에 시간이 걸릴 수 있음